<a href="https://colab.research.google.com/github/willjack85/framepack-collab/blob/main/FramePack_colab_SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [stable-diffusion-art.com](https://stable-diffusion-art.com) - Check out latest tutorials.

### [Guide to use this notebook](https://stable-diffusion-art.com/framepack-colab/) - Leave comment if you have questions.


#### <b>Become a member to support my work.</b>

[<img src="https://stable-diffusion-art.com/wp-content/uploads/2023/10/see_member_benefit_button300.png" width=200>](https://stable-diffusion-art.com/member)


This notebook launches [FramePack](https://github.com/lllyasviel/FramePack).

## Update
- 5/06 2025: Added F1 support.
- 5/03/2025: Fixed ngrok support...
- 5/02/2025: Added ngrok support.
- 4/29/2025: First version.


In [ ]:
#@title <font size="6" color="orange">FramePack</font>
#@markdown #### <--- Click to run. Follow the **public** URL link **`https://xxxxxxx.gradio.live`** to launch the app.
FramePack_F1=False #@param {type: "boolean"}
#@markdown [FramePack-F1](https://github.com/lllyasviel/FramePack/discussions/459) predicts future frames from historical frames and produces more varieties.
NGROK='' #@param {type: "string"}
#@markdown Use NGROK if Gradio has connection issues. Get API key [here](https://dashboard.ngrok.com/get-started/your-authtoken). You can also use [notebook secret](https://stable-diffusion-art.com/comfyui-colab/#Notebook_secrets): `NGROK`.

# Save_Videos_in_Google_Drive = True #@param{type: "boolean"}
# Save_folder = 'AI_PICS/FramePack' #@param {type:"string"}

def ngrok_thread(port):
  %pip install pyngrok
  # from pyngrok import ngrok, conf
  # conf.get_default().config_path = "/content/ngrok.yaml".  # pyngrok doesn't support traffic policy https://github.com/alexdlaird/pyngrok/issues/138
  # conf.get_default().config_version = "3"

  PASSWORD='stablediffusionart'
  import time
  import socket

  # Wait for the app to open port
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()

  # Write config
  NGROK_CONFIG=f'''version: 3
endpoints:
    - name: gradio
      upstream:
        url: http://localhost:{port}
      traffic_policy:
        inbound:
            - name: "Remove X-Forwarded-Host"
              actions:
                - type: remove-headers
                  config:
                    headers:
                        - "x-forwarded-host"
agent:
    authtoken: {NGROK}
  '''
  with open("/content/ngrok_config.yaml", "w") as f:
    f.write(NGROK_CONFIG)

  !ngrok start gradio --config /content/ngrok_config.yaml --log=stdout

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Python 3.10 detected. Skip installation.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def clear():
    from IPython.display import clear_output; return clear_output()


# Get secrets
from google.colab import userdata
try:
  NGROK = userdata.get('NGROK')
  print('Used NGROK secret.')
except:
  pass

# ngrok
#if NGROK:

# save in google drive
# if Save_Videos_in_Google_Drive:
#   import os
#   from google.colab import drive
#   drive.mount('/content/drive')

# install python 3.10
updatePython()

# install FramePack
%cd /content/
!git clone https://github.com/lllyasviel/FramePack
%cd FramePack
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install -r requirements.txt
!pip install sageattention==1.0.6
# if Save_Videos_in_Google_Drive:
#     output_path = '/content/drive/MyDrive/' + Save_folder
#     if not os.path.exists(output_path):
#       %mkdir -p {output_path}
#     !rm -rf /content/FramePack/outputs/
#     !ln -s {output_path} /content/FramePack/outputs



# Run
cmd = 'PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True'
if FramePack_F1:
  cmd += ' python demo_gradio_f1.py'
else:
  cmd += ' python demo_gradio.py'

clear()
if NGROK:
  import threading
  # ngrok
  port = 7860
  # if not 'ngrok_listener' in locals():
  #   ngrok_listener = ngrok_connect(port)
  threading.Thread(target=ngrok_thread, daemon=True, args=(port,)).start()
else:
  cmd += ' --share'

print(cmd)
!{cmd}